In [ ]:
!pip install playwright
!pip install playwright nest_asyncio
!playwright install


Playwright Host validation warning: 
╔══════════════════════════════════════════════════════╗
║ Host system is missing dependencies to run browsers. ║
║ Missing libraries:                                   ║
║     libwoff2dec.so.1.0.2                             ║
║     libgstgl-1.0.so.0                                ║
║     libgstcodecparsers-1.0.so.0                      ║
║     libharfbuzz-icu.so.0                             ║
║     libenchant-2.so.2                                ║
║     libsecret-1.so.0                                 ║
║     libhyphen.so.0                                   ║
║     libmanette-0.2.so.0                              ║
╚══════════════════════════════════════════════════════╝
    at validateDependenciesLinux (/usr/local/lib/python3.10/dist-packages/playwright/driver/package/lib/server/registry/dependencies.js:216:9)
    at process.processTicksAndRejections (node:internal/process/task_queues:95:5)
    at async Registry._validateHostRequirements (/usr/

In [ ]:
import nest_asyncio
import asyncio
import pandas as pd
from playwright.async_api import async_playwright
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive')

# Colab 환경에서 실행 중인 이벤트 루프를 허용
nest_asyncio.apply()

async def run():
    reviews = []
    async with async_playwright() as p:
        # Chromium 브라우저 시작
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        # 마켓컬리 상품 페이지로 이동
        base_url = "https://www.kurly.com/goods/5050290"
        try:
            await page.goto(base_url, timeout=0)  # 타임아웃을 무한정으로 설정

            # 페이지가 특정 요소를 로드할 때까지 기다리기
            await page.wait_for_selector("p.css-y49dcn.e36z05c13", timeout=60000)

            # 페이지네이션을 통한 리뷰 추출
            page_number = 1
            while True:
                # 리뷰 추출 - 마켓컬리 페이지의 HTML 구조에 따라 적절하게 선택자 설정
                review_elements = await page.query_selector_all("p.css-y49dcn.e36z05c13")

                if not review_elements:
                    print(f"{page_number} 페이지에 더 이상 리뷰가 없습니다. 종료합니다.")
                    break  # 더 이상 리뷰가 없는 경우 종료

                for element in review_elements:
                    review_text = await element.inner_text()
                    label = 1 if "맛있다" in review_text or "좋다" in review_text else 0

                    reviews.append({
                        "상품평": review_text,
                        "label": label
                    })

                # 진행 상황을 주기적으로 저장
                review_df = pd.DataFrame(reviews)
                review_df.to_csv('/content/drive/MyDrive/crawled_reviews_dataset.csv', index=False, encoding='utf-8-sig')

                # 다음 페이지로 이동
                page_number += 1
                next_page_url = f"{base_url}?page={page_number}"
                try:
                    await page.goto(next_page_url, timeout=0)
                    await page.wait_for_selector("p.css-y49dcn.e36z05c13", timeout=60000)  # 다음 페이지가 로드될 때까지 대기
                except Exception as e:
                    print(f"{page_number} 페이지로 이동 중 오류 발생: {e}. 종료합니다.")
                    break

        except Exception as e:
            print(f"페이지 로딩 중 오류 발생: {e}")
        finally:
            # 브라우저 종료
            await browser.close()

    # 최종적으로 데이터프레임으로 변환 및 저장
    if reviews:
        review_df = pd.DataFrame(reviews)
        review_df.to_csv('/content/drive/MyDrive/crawled_reviews_dataset.csv', index=False, encoding='utf-8-sig')
        print("리뷰 크롤링 및 저장이 완료되었습니다.")
    else:
        print("리뷰 데이터가 수집되지 않았습니다.")

# asyncio 이벤트 루프 실행
try:
    asyncio.run(run())
except RuntimeError as e:
    print(f"런타임 오류 발생: {e}. 이미 실행 중인 이벤트 루프를 감지하여 nest_asyncio를 사용해 다시 시도합니다.")
    nest_asyncio.apply()
    asyncio.get_event_loop().run_until_complete(run())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/pyee/base.py:186: RuntimeWarning: coroutine 'run' was never awaited
  funcs = list(self._events.get(event, OrderedDict()).values())
ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-5' coro=<run() done, defined at <ipython-input-9-ee3c93504f0f>:8> exception=Exception('Browser.close: Connection closed while reading from the driver')>
Traceback (most recent call last):
  File "/usr/lib/python3.10/asyncio/tasks.py", line 232, in __step
    result = coro.send(None)
  File "<ipython-input-9-ee3c93504f0f>", line 56, in run
    await browser.close()
  File "/usr/local/lib/python3.10/dist-packages/playwright/async_api/_generated.py", line 14173, in close
    return mapping.from_maybe_impl(await self._impl_obj.close(reason=reason))
  File "/usr/local/lib/python3.10/dist-packages/playwright/_impl/_browser.py", line 192, in close
    raise e
  File "/usr/local/lib/python3.10/dist-packages/playwright/_impl/_browser.py", line 

Exception: Browser.close: Connection closed while reading from the driver